In [9]:
# Fix seeds to control for randomly varying outputs (due to selection of equally-frequent words in a random fashion)
import os
os.environ['PYTHONHASHSEED']=str(0)

import random
random.seed(0)

import numpy as np
np.random.seed(0)

In [10]:
import nltk
import re
import heapq
from utils import *         # helper functions like get_word2count, get_word2idx_and_idx2word,  etc.  
import time
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

### Import the dataset

In [11]:
from nltk.corpus import brown
from nltk.tokenize.moses import MosesDetokenizer

# Use a detokenizer to remove the annotations/special tokens/tags from the brown corpus
mdetok = MosesDetokenizer()

dataset = [mdetok.detokenize(' '.join(sent).replace('``', '"').replace("''", '"').replace('`', "'").split(), return_str=True)  for sent in brown.sents()]
print('Total sentences: ',len(dataset))
random.shuffle(dataset)
dataset[0:4]

Total sentences:  57340


['Stars for marriage',
 'During the first pass of Phase 3,, references to the actual addresses of index words and electronic switches are collected and the availability table is updated.',
 'As we expected, on the following day my Uncle was completely recovered and opened the store as usual at 10 in the morning.',
 'Books are not the only resource of the system.']

### Preprocessing

In [12]:
print(dataset[1450])
sentences = preprocess(dataset)[:10000] # keep first 10k sentences only to allow for faster run-time.
print(sentences[1450])

He had just paid a brief visit to the Frick Collection to admire his favorite paintings by Rembrandt and Franz Hals.
he had just paid a brief visit to the frick collection to admire his favorite paintings by rembrandt and franz hals.


### Count word occurences & prepare histogram of top-10000 words

In [13]:
# Histogram of occurences of words in the dataset
word2count = get_word2count(sentences)
print(len(word2count))

19181


In [14]:
V = 10000                                     # No. of words in vocabulary, i.e. vocabulary size

freq_words = heapq.nlargest(V, word2count, key=word2count.get)  # Select only "V" most frequent words
print('# of words selected: ',V,'\n')

# Print first 3 and last 3 words with their occurence rate
for i in range(3):
    print('{}   :   {} '.format(freq_words[i],word2count[freq_words[i]]))  
for i in range(3):
    print('{}   :   {} '.format(freq_words[-i-1],word2count[freq_words[-i-1]]))  

# of words selected:  5000 

the   :   12251 
.   :   10000 
of   :   6301 
mosque   :   4 
purdew   :   4 
simms   :   4 


### Map each word to a unique number (index)

In [15]:
# Mapping most frequent words in the dataset to a unique index. This is used to translate sentences consisting of words to a list of numbers (indexes).
word2idx, idx2word = get_word2idx_and_idx2word(sentences, freq_words)

# Update V variable to include any extra tokens like ["START","END","UNKNOWN"] added in word2idx
V = len(idx2word)

print('No. of words in vocabulary: {}'.format(V))

# Print a few examples of mappings
sample_words = random.sample(freq_words, 6)
print(sample_words)

print(word2idx[sample_words[0]], word2idx[sample_words[1]], word2idx[sample_words[2]], word2idx[sample_words[3]],word2idx[sample_words[4]],word2idx[sample_words[5]])

# Verify that the mappings are correct
print(idx2word[word2idx[sample_words[0]]], idx2word[word2idx[sample_words[1]]], idx2word[word2idx[sample_words[2]]], idx2word[word2idx[sample_words[3]]],idx2word[word2idx[sample_words[4]]],idx2word[word2idx[sample_words[5]]])

No. of words in vocabulary: 5003
['determination', 'knife', 'hardly', 'glance', 'herd', 'satisfy']
935 2247 649 987 2498 4694
determination knife hardly glance herd satisfy


### Transform all the sentences using these {word:index} mappings

In [17]:
new_sentences = transform_sentences(sentences, word2idx)

# Print the same sentence from the "Preprocessing" part in new form
print('Original: ',sentences[1450])
print('Transformed: ',new_sentences[1450])
print([idx2word[idx] for idx in new_sentences[1450]])

Original:  he had just paid a brief visit to the frick collection to admire his favorite paintings by rembrandt and franz hals.
Transformed:  [0, 99, 75, 341, 659, 85, 1412, 1998, 12, 7, 5002, 1024, 12, 5002, 101, 3637, 2458, 188, 5002, 16, 5002, 5002, 5, 1]
['START', 'he', 'had', 'just', 'paid', 'a', 'brief', 'visit', 'to', 'the', 'UNKNOWN', 'collection', 'to', 'UNKNOWN', 'his', 'favorite', 'paintings', 'by', 'UNKNOWN', 'and', 'UNKNOWN', 'UNKNOWN', '.', 'END']


## 1. Counting method

In [18]:
# Counting method bigram-model
def get_bigram_probs(sentences, word2idx, vocab_size):
    """Convert each sentence in "sentences" from a collection of words to a collection of numbers, where a number is the unique index the original word was mapped to.
    Arguments:
    sentences -- list, length N, each element containing numerical format of corresponding sentence in the dataset.
    word2idx -- dictionary, with {word: mapped_index} as key:value pairs.
    vocab_size -- int, number of words in the vocabulary
    Returns:
    bigram_probs -- numpy.ndarray, shape-(vocab_size, vocab_size), contains the probability distributions for next word (along axis 1, i.e. columns) given current word (along axis 0, i.e. rows)."""
    
    # not initializing with zeros so that there is some minimalistic probability for all bigrams.
    bigram_matrix = np.ones((vocab_size, vocab_size))  
    
    for sentence in sentences:
        for i in range(1, len(sentence)):
            bigram_matrix[sentence[i-1], sentence[i]] += 1
    bigram_matrix /= bigram_matrix.sum(axis=1, keepdims=True)
    return bigram_matrix

#### Training

In [19]:
# Learn probability distribution of bi-grams in the "new_sentences" dataset using above function
bigram_probs = get_bigram_probs(new_sentences, word2idx, vocab_size=V)

#### Testing

In [20]:
# 1. Sentence scores: the log-probability (normalized by length of sentence) of a sentence being picked among the universe of sentences possible using top-1500 words of the dataset.
print('Task 1.')
# Test on a sentence from the dataset
print("Sentence: {}         Score: {}".format(inv_transform_sentence(new_sentences[34], idx2word), '%.4f'%get_sentence_score(new_sentences[34], bigram_probs)) )

# Test on a correct new sentence
test_sentence = transform_sentences(["This is one of the most exciting events people choose to go to."], word2idx)
print("Sentence: {}         Score: {}".format(inv_transform_sentence(test_sentence[0], idx2word), '%.4f'%get_sentence_score(test_sentence[0], bigram_probs)) )

# Test on an incorrect new sentence
test_sentence = transform_sentences(["Church kind like I about care interesting suddenly"], word2idx)
print("Sentence: {}         Score: {}".format(inv_transform_sentence(test_sentence[0], idx2word), '%.4f'%get_sentence_score(test_sentence[0], bigram_probs)) )

# Add the .("dot") at the end of the above sentence to see the score improving a bit.
test_sentence = transform_sentences(["Church kind like I about care interesting suddenly."], word2idx)
print("Sentence: {}         Score: {}\n\n".format(inv_transform_sentence(test_sentence[0], idx2word), '%.4f'%get_sentence_score(test_sentence[0], bigram_probs)) )


# 2. Sentence completion: completion of a sentence given some initial words using the probability distribution learned by the counting-method bigram model
print('Task 2.')
print(generate_sentence("Law of the state", bigram_probs, word2idx, idx2word, max_len=20) )
print(generate_sentence("High tax", bigram_probs, word2idx, idx2word, max_len=20) )
print(generate_sentence("There were reports", bigram_probs, word2idx, idx2word, max_len=20) )

Task 1.
Sentence: the UNKNOWN data given include UNKNOWN UNKNOWN and unit cell dimensions .         Score: -5.1772
Sentence: this is one of the most exciting events people choose to go to .         Score: -5.2647
Sentence: church kind like i about care interesting suddenly         Score: -7.6008
Sentence: church kind like i about care interesting suddenly .         Score: -6.8837


Task 2.
Law of the state constitution tile goods projects control powder station trust montero foster woods monday swung regional miss plato new thinks cultures auto
High tax lodge prime built composed scientists trade vote morris fifteenth levels directly cultures collective exchange mirror escort accordingly holder herbert installed
There were reports red hundred chapter scene conflict toes whispered propose aside symbolic chaos nurse intent information promises mental anyway march louis turn


## 2. Simple Neural Network
##### (Only Softmax layer, no hidden layer)

In [13]:
len(new_sentences)

10000

In [21]:
W = np.random.uniform(size=(V,V))        # Initialize the weights of the network

# Specify Hyperparameters for the network
epochs = 1
lr = 0.01

loss_hist = []
training_start_time = time.time()
for epoch in range(epochs):
    epoch_start_time = time.time()
    for i, sentence in enumerate(new_sentences):
        n = len(sentence)
        
        # convert each word of the sentence to one-hot-encoded format
        inp_sentence = np.zeros((n, V))
        inp_sentence[np.arange(n), sentence] = 1  
        
        X = inp_sentence[:n-1, :]              # shape: N x V
        y = inp_sentence[1:, :].T              # shape: V x N
        
        preds = softmax(np.dot(W, X.T))    # 1. Forward propagation     shape: V x N
        grads = np.dot((preds - y), X)     # 2. Compute gradients       shape: V x N
        
        W = W - lr* grads                  # 3. Gradient descent step
        
        loss = - np.sum(y*np.log(preds), axis=None)/n-1
        loss_hist.append(loss)
        
        if i%50==49:
            print('Epoch: {}/{}   --   Sentence: {}/{}   --   Loss: {}'.format(epoch+1, epochs, '%5.0f'%(i+1), '%5.0f'%len(new_sentences), '%.4f'%loss), end='\r')
    print('Epoch {} complete!!   time_elapsed: {}s'.format(epoch+1, '%.2f'%(time.time()-epoch_start_time)))

print('Training complete!!   time_elapsed: {}s'.format('%.2f'%(time.time()-training_start_time)))

Epoch 1 complete!!   time_elapsed: 2836.61s-   Loss: 6.0573
Training complete!!   time_elapsed: 2836.61s


#### Testing

In [15]:
# 1. Sentence scores: the log-probability (normalized by length of sentence) of a sentence being picked among the universe of sentences possible using top-1500 words of the dataset.
print('Task 1.')
# Test on a sentence from the dataset
print("Sentence: {}         Score: {}".format(inv_transform_sentence(new_sentences[34], idx2word), '%.4f'%get_sentence_score_nn1(new_sentences[34], W)) )

# Test on a correct new sentence
test_sentence = transform_sentences(["This is one of the most exciting events people choose to go to."], word2idx)
print("Sentence: {}         Score: {}".format(inv_transform_sentence(test_sentence[0], idx2word), '%.4f'%get_sentence_score_nn1(test_sentence[0], W)) )

# Test on an incorrect new sentence
test_sentence = transform_sentences(["Church kind like I about care interesting suddenly"], word2idx)
print("Sentence: {}         Score: {}".format(inv_transform_sentence(test_sentence[0], idx2word), '%.4f'%get_sentence_score_nn1(test_sentence[0], W)) )

# Add the .("dot") at the end of the above sentence to see the score improving a bit.
test_sentence = transform_sentences(["Church kind like I about care interesting suddenly."], word2idx)
print("Sentence: {}         Score: {}\n\n".format(inv_transform_sentence(test_sentence[0], idx2word), '%.4f'%get_sentence_score_nn1(test_sentence[0], W)) )


# 2. Sentence completion: completion of a sentence given some initial words using the probability distribution learned by the counting-method bigram model
print('Task 2.')
print(generate_sentence_nn1("Law of the state",   W, word2idx, idx2word, max_len=20) )
print(generate_sentence_nn1("High tax",           W, word2idx, idx2word, max_len=20) )
print(generate_sentence_nn1("There were reports", W, word2idx, idx2word, max_len=20) )

Task 1.
Sentence: the UNKNOWN data given include UNKNOWN UNKNOWN and unit cell dimensions .         Score: -6.6371
Sentence: this is one of the most exciting events people choose to go to .         Score: -8.0741
Sentence: church kind like i about care interesting suddenly         Score: -7.8909
Sentence: church kind like i about care interesting suddenly .         Score: -7.9584


Task 2.
Law of the state does early identified national inside elbow rare anonymous activation efforts spare cathy ham pull earnestly before conclusion convinced employees men
High tax bureau control their fence workers later floors ruth missing banks fun freezing drunk sat fair planning guarantee editors patients we
There were reports fromm attitude principle site board attributed rhode frames mouth methodist laboratory prize ignore equipment leadership negro print consumer proposals grinned


## 3. Neural Network with 1 hidden layer

In [22]:
# Specify Hyperparameters for the network
D = 300                                     # hidden layer size
epochs = 5
lr = 0.001

W1 = np.random.uniform(size=(D, V))        # Initialize the weights of the network
W2 = np.random.uniform(size=(V, D))        # Initialize the weights of the network


loss_hist = []
training_start_time = time.time()
for epoch in range(epochs):
    epoch_start_time = time.time()
    for i, sentence in enumerate(new_sentences):
        n = len(sentence)
        
        # convert each word of the sentence to one-hot-encoded format
        inp_sentence = np.zeros((n, V))
        inp_sentence[np.arange(n), sentence] = 1  
        
        X = inp_sentence[:n-1, :].T            # shape: V x N
        y = inp_sentence[1:, :].T              # shape: V x N
        
        # 1. Forward propagation
        z1 = np.dot(W1, X)                     # shape: D x N
        a1 = z1 * (z1>0)                       # shape: D x N
        
        z2 = np.dot(W2, a1)                    # shape: V x N
        preds = softmax(z2)                    # shape: V x N
        
        # 2. Loss calculation
        loss = - np.sum(y*np.log(preds), axis=None)/n-1   # float, single number
        
        # 3. Back propagation
        grads_z2 = preds - y                   # shape: V x N
        grads_W2 = np.dot(grads_z2, a1.T)      # shape: V x D
        
        grads_a1 = np.dot(W2.T, grads_z2)      # shape: D x N
        grads_z1 = grads_a1 * (a1>0)           # shape: D x N
        grads_W1 = np.dot(grads_z1, X.T)       # shape: D x V
        
        # 4. Update weights
        W2 = W2 - lr* grads_W2 
        W1 = W1 - lr* grads_W1 
        
        loss_hist.append(loss)
        
        if i%50==49:
            print('Epoch: {}/{}   --   Sentence: {}/{}   --   Loss: {}'.format(epoch+1, epochs, '%5.0f'%(i+1), '%5.0f'%len(new_sentences), '%.4f'%loss), end='\r')
    print('Epoch {} complete!!   time_elapsed: {}s'.format(epoch+1, '%.2f'%(time.time()-epoch_start_time)))

print('Training complete!!   time_elapsed: {}s'.format('%.2f'%(time.time()-training_start_time)))

Epoch 1 complete!!   time_elapsed: 206.16s--   Loss: 4.9309
Epoch 2 complete!!   time_elapsed: 204.17s--   Loss: 4.7516
Epoch 3 complete!!   time_elapsed: 205.16s--   Loss: 4.6671
Epoch 4 complete!!   time_elapsed: 210.69s--   Loss: 4.6085
Epoch 5 complete!!   time_elapsed: 211.47s--   Loss: 4.5597
Training complete!!   time_elapsed: 1037.65s


#### Testing

In [28]:
# 1. Sentence scores: the log-probability (normalized by length of sentence) of a sentence being picked among the universe of sentences possible using top-1500 words of the dataset.
print('Task 1.')
# Test on a sentence from the dataset
print("Sentence: {}         Score: {}".format(inv_transform_sentence(new_sentences[34], idx2word), '%.4f'%get_sentence_score_nn2(new_sentences[34], W1, W2)) )

# Test on a correct new sentence
test_sentence = transform_sentences(["This is one of the most exciting events people choose to go to."], word2idx)
print("Sentence: {}         Score: {}".format(inv_transform_sentence(test_sentence[0], idx2word), '%.4f'%get_sentence_score_nn2(test_sentence[0], W1, W2)) )

# Test on an incorrect new sentence
test_sentence = transform_sentences(["Church kind like I about care interesting suddenly"], word2idx)
print("Sentence: {}         Score: {}".format(inv_transform_sentence(test_sentence[0], idx2word), '%.4f'%get_sentence_score_nn2(test_sentence[0], W1, W2)) )

# Add the .("dot") at the end of the above sentence to see the score improving a bit.
test_sentence = transform_sentences(["Church kind like I about care interesting suddenly."], word2idx)
print("Sentence: {}         Score: {}\n\n".format(inv_transform_sentence(test_sentence[0], idx2word), '%.4f'%get_sentence_score_nn2(test_sentence[0], W1, W2)) )


# 2. Sentence completion: completion of a sentence given some initial words using the probability distribution learned by the counting-method bigram model
print('Task 2.')
print(generate_sentence_nn2("Law of the state",   W1, W2, word2idx, idx2word, max_len=20) )
print(generate_sentence_nn2("High tax",           W1, W2, word2idx, idx2word, max_len=20) )
print(generate_sentence_nn2("There were reports", W1, W2, word2idx, idx2word, max_len=20) )

Task 1.
Sentence: the UNKNOWN data given include UNKNOWN UNKNOWN and unit cell dimensions .         Score: -7.6060
Sentence: this is one of the most exciting events people choose to go to .         Score: -7.6519
Sentence: church kind like i about care interesting suddenly         Score: -8.4152
Sentence: church kind like i about care interesting suddenly .         Score: -8.5742


Task 2.
Law of the state was in introduction by is UNKNOWN has UNKNOWN you member he and mock af the af and more signal and
High tax of and avoid and for taken and of him of jazz of he would visit action the why of far
There were reports of from . will for UNKNOWN . eliminated more without something . us UNKNOWN . for as by to immediate
